In [13]:
#import package
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from model import RNN,LSTM,GRU
import torch.nn.functional as F

In [2]:
# def pad_time_series(data, target_length=None, padding_value=0.0):
#     # Determine the target length if not provided
#     if target_length is None:
#         target_length = max(len(sample) for sample in data)

#     for sample in data:
#         if len(sample) < target_length:
#             for _ in range(target_length - len(sample)):
#                 sample.append([padding_value for _ in range(4)])

#     return data

# # Example usage:
# data = [
#     [[0.1, 0.2, 0.3, 0.5], [0.1, 0.2, 0.3, 0.5], [0.1, 0.2, 0.3, 0.5], [0.1, 0.2, 0.3, 0.5], [0.1, 0.2, 0.3, 0.5]],   #report1 representation
#     [[0.1, 0.2, 0.3, 0.5], [0.1, 0.2, 0.3, 0.5]]                                                                     #report2 representation
# ]

# target_length = 6
# padded_data = pad_time_series(data, target_length, padding_value=0.25)
# print(padded_data)


[[[0.1, 0.2, 0.3, 0.5], [0.1, 0.2, 0.3, 0.5], [0.1, 0.2, 0.3, 0.5], [0.1, 0.2, 0.3, 0.5], [0.1, 0.2, 0.3, 0.5], [0.25, 0.25, 0.25, 0.25]], [[0.1, 0.2, 0.3, 0.5], [0.1, 0.2, 0.3, 0.5], [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25]]]


In [23]:
data = pd.read_csv('C:/Users/lenovo/Desktop/UCL/Final dissertation/code/ESGOverallData.csv')
ESG_data = data['ESGData'].apply(eval)
length = ESG_data.apply(len)


def pad_time_series(data, target_length=None, padding_value=0.25):
    if target_length is None:
        target_length = length.mean()
    
    padded_data = []
    for sample, length in zip(data, length):
        current_length = len(sample)
        if current_length < target_length:
            sample += [[padding_value for _ in range(4)] for _ in range(int(target_length - current_length))]
        elif current_length > target_length:
            sample = sample[:int(target_length)]
        padded_data.append(sample)

    return padded_data

In [ ]:
padded_data = pad_time_series(data, target_length, padding_value=0.25)
print(padded_data)

In [ ]:
def pad_time_series(data, target_length=None, padding_value=0.25):
    # Determine the target length if not provided
    if target_length is None:
        target_length = max(len(sample) for sample in data)

    for sample in data:
        if len(sample) < target_length:
            for _ in range(target_length - len(sample)):
                sample.append([padding_value for _ in range(4)])

    return data

In [2]:
#regression dataset
class MyDataset1(data.Dataset):
    def __init__(self, X, Y):
        self.samples = X
        self.labels = Y

    def __getitem__(self, index):
        sample = self.samples[index]
        label = [self.labels[index]]
        #print(label)
        sample=torch.Tensor(sample).float()
        label= torch.Tensor(label).float()

        return sample, label

    def __len__(self):
        return len(self.samples)

In [8]:
#training loop
def loop(model,train_loader,test_loader,optimizer,criterion,device,E=20):
    loss_all=[]
    for epoch in range(E): 
        loss_epoch=[]
        print(epoch)
        for samples, labels in train_loader:
            samples=samples.to(device)
            labels=labels.to(device)
            #print(samples.shape)
            #print(labels.shape)
        # Set the flag to training mode
            model.train()
            
            optimizer.zero_grad()
            outputs = model(samples)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            loss_epoch.append(float(loss.cpu()))
            #print(outputs)
        print(sum(loss_epoch)/len(loss_epoch))
        loss_all.append(sum(loss_epoch)/len(loss_epoch))

    model.eval()
    result=[]
    for samples, true in test_loader:
        samples=samples.to(device)
        outputs = model(samples)
        result.append(float(outputs.cpu()))
        
    return result,loss_epoch,model

In [9]:
x=[[[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2]],
    [[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2]],
    [[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2]],
    [[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2]],
    [[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2]],
    [[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2]],
    [[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2]],
    [[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2]],
    [[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2]],
    [[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2]],
    [[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2]],
    [[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2]],
    [[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2]],
    [[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2]],
    [[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2]],
    [[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2],[0.3,0.4,0.1,0.2]]]
y=[1,2,3,4,5,6,7,8,9,0,1,2,3,4,5,6]

In [10]:
#parameters
batch_size=4
learning_rate=0.000001
epoch=150
hidden_num=50
output_dim=1 #if None put ''
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
train_dataset = MyDataset1(x,y)
train_loader=data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader=data.DataLoader(dataset=train_dataset, batch_size=1, shuffle=True)
criterion = nn.MSELoss()

In [12]:
features_dim=np.array(x).shape[2]
modelRNN=RNN.RNN_Model(features_dim,hidden_num,output_dim).to(device)
optimizerRNN= optim.Adam(modelRNN.parameters(), lr=learning_rate)
resultRNN,lossRNN,modelRNN=loop(modelRNN,train_loader,test_loader,optimizerRNN,criterion,device,E=epoch)

0
23.935096263885498
1
23.932191371917725
2
23.929420471191406
3
23.926559925079346
4
23.923686981201172
5
23.921006679534912
6
23.91792106628418
7


d:\anaconda\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([4, 1])) that is different to the input size (torch.Size([4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


23.91513729095459
8
23.912482023239136
9
23.909560680389404
10
23.90682554244995
11
23.904036045074463
12
23.901008367538452
13
23.89863157272339
14
23.895607948303223
15
23.892842769622803
16
23.88995122909546
17
23.88701105117798
18
23.88428223133087
19
23.88149917125702
20
23.87871241569519
21
23.876052141189575
22
23.873072624206543
23
23.870450496673584
24
23.867482900619507
25
23.864731311798096
26
23.86180090904236
27
23.859086275100708
28
23.85630965232849
29
23.853467106819153
30
23.85075044631958
31
23.848214387893677
32
23.84511947631836
33
23.842270851135254
34
23.83950400352478
35
23.83678913116455
36
23.834123373031616
37
23.831236839294434
38
23.828282833099365
39
23.825486660003662
40
23.8226780295372
41
23.819843530654907
42
23.817372798919678
43
23.814438819885254
44
23.811674118041992
45
23.808974981307983
46
23.806119441986084
47
23.80320167541504
48
23.800493240356445
49
23.797857403755188
50
23.795089960098267
51
23.791996479034424
52
23.78947901725769
53
23.78665